In [1]:
import pandas as pd
import re
import tensorflow as tf
import numpy as np
from keras.utils.np_utils import to_categorical

data = pd.read_csv("./yelp_labelled.txt", delimiter='\t', header=None, names=['review','label'], encoding='utf-8')

data.shape

/Users/jiangs19/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/jiangs19/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


(1000, 2)

In [2]:
data.head(2)

,review,label
0,Wow... Loved this place.,1
1,Crust is not good.,0


### CNN starts here...

In [3]:
docs = []
sentences = []
sentiments = []

for sentences, sentiment in zip(data.review, data.label):
    sentences_cleaned = [sent.lower() for sent in sentences]
    docs.append(sentences_cleaned)
    sentiments.append(sentiment)

len(docs), len(sentiments)

(1000, 1000)

In [15]:
docs[0]

['w',
 'o',
 'w',
 '.',
 '.',
 '.',
 ' ',
 'l',
 'o',
 'v',
 'e',
 'd',
 ' ',
 't',
 'h',
 'i',
 's',
 ' ',
 'p',
 'l',
 'a',
 'c',
 'e',
 '.']

In [19]:
len(sentences)

134

In [17]:
sentiments[0]

1

In [4]:
maxlen = 1024 
nb_filter = 256
dense_outputs = 1024
filter_kernels = [7, 7, 3, 3, 3, 3]
n_out = 2
batch_size = 80
nb_epoch = 10

In [5]:
txt = ''
for doc in docs:
    for s in doc:
        txt += s
chars = set(txt)
vocab_size = len(chars)
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 56


In [20]:
chars

{' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'é',
 'ê'}

In [6]:
from keras.preprocessing.sequence import pad_sequences

def vectorize_sentences(data, char_indices):
    X = []
    for sentences in data:
        x = [char_indices[w] for w in sentences]
        x2 = np.eye(len(char_indices))[x]
        X.append(x2)
    return (pad_sequences(X, maxlen=maxlen))

train_data = vectorize_sentences(docs,char_indices)
train_data.shape
y_train = to_categorical(sentiments)

In [10]:
train_data[0].shape

(1024, 56)

In [14]:
train_data.shape

(1000, 1024, 56)

In [21]:
y_train.shape

(1000, 2)

In [25]:
y_train[:3]

array([[0., 1.],
       [1., 0.],
       [1., 0.]])

In [28]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution1D, MaxPooling1D

inputs = Input(shape=(maxlen, vocab_size), name='input', dtype='float32')

conv = Convolution1D(nb_filter=nb_filter, filter_length=filter_kernels[0],
                     border_mode='valid', activation='relu',
                     input_shape=(maxlen, vocab_size))(inputs)
conv = MaxPooling1D(pool_length=3)(conv)

conv1 = Convolution1D(nb_filter=nb_filter, filter_length=filter_kernels[1],
                      border_mode='valid', activation='relu')(conv)
conv1 = MaxPooling1D(pool_length=3)(conv1)

conv2 = Convolution1D(nb_filter=nb_filter, filter_length=filter_kernels[2],
                      border_mode='valid', activation='relu')(conv1)

conv3 = Convolution1D(nb_filter=nb_filter, filter_length=filter_kernels[3],
                      border_mode='valid', activation='relu')(conv2)

conv4 = Convolution1D(nb_filter=nb_filter, filter_length=filter_kernels[4],
                      border_mode='valid', activation='relu')(conv3)

conv5 = Convolution1D(nb_filter=nb_filter, filter_length=filter_kernels[5],
                      border_mode='valid', activation='relu')(conv4)
conv5 = MaxPooling1D(pool_length=3)(conv5)
conv5 = Flatten()(conv5)

z = Dropout(0.5)(Dense(dense_outputs, activation='relu')(conv5))
z = Dropout(0.5)(Dense(dense_outputs, activation='relu')(z))

pred = Dense(n_out, activation='softmax', name='output')(z)

model = Model(input=inputs, output=pred)

model.compile(loss='categorical_crossentropy', optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

/Users/jiangs19/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", input_shape=(1024, 56), filters=256, kernel_size=7, padding="valid")`
  if __name__ == '__main__':
/Users/jiangs19/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=3)`
  # Remove the CWD from sys.path while we load stuff.
/Users/jiangs19/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=256, kernel_size=7, padding="valid")`
  del sys.path[0]
/Users/jiangs19/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=3)`
  
/Users/jiangs19/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Update you

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 1024, 56)          0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 1018, 256)         100608    
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 339, 256)          0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 333, 256)          459008    
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 111, 256)          0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 109, 256)          196864    
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 107, 256)          196864    
__________

### Word based CNN

In [39]:
data = pd.read_csv("./yelp_labelled.txt",
                   delimiter='\t', header=None, names=['review','label'],encoding='utf-8')
# z = list(nlp.pipe(data['review'], n_threads=20, batch_size=20000))

from __future__ import unicode_literals
from spacy.lang.en import English
from collections import Counter

raw_text = 'Hello, world. Here are two sentences.'
nlp = English()

def tokenizeSentences(sent):
    doc = nlp(sent)
    sentences = [sent.string.strip() for sent in doc]
    return sentences

Xs = []    
for texts in data['review']:
    Xs.append(tokenizeSentences(texts))

Xs[:5]

[['Wow', '...', 'Loved', 'this', 'place', '.'],
 ['Crust', 'is', 'not', 'good', '.'],
 ['Not', 'tasty', 'and', 'the', 'texture', 'was', 'just', 'nasty', '.'],
 ['Stopped',
  'by',
  'during',
  'the',
  'late',
  'May',
  'bank',
  'holiday',
  'off',
  'Rick',
  'Steve',
  'recommendation',
  'and',
  'loved',
  'it',
  '.'],
 ['The',
  'selection',
  'on',
  'the',
  'menu',
  'was',
  'great',
  'and',
  'so',
  'were',
  'the',
  'prices',
  '.']]

In [42]:
from functools import reduce
vocab = sorted(reduce(lambda x, y: x | y, (set(words) for words in Xs)))
len(vocab)

2349

In [43]:
import operator

def word_freq(Xs, num):
    all_words = [words.lower() for sentences in Xs for words in sentences]
    sorted_vocab = sorted(dict(Counter(all_words)).items(), key=operator.itemgetter(1))
    final_vocab = [k for k,v in sorted_vocab if v>num]
    word_idx = dict((c, i + 1) for i, c in enumerate(final_vocab))
    return final_vocab, word_idx

final_vocab, word_idx = word_freq(Xs,2)
vocab_len = len(final_vocab) # Finally we have 598 words!

In [44]:
def vectorize_sentences(data, word_idx, final_vocab, maxlen=40):
    X = []
    paddingIdx = len(final_vocab)+2
    for sentences in data:
        x=[]
        for word in sentences:
            if word in final_vocab:
                x.append(word_idx[word])
            elif word.lower() in final_vocab:
                x.append(word_idx[word.lower()])
            else:
                x.append(paddingIdx)
        X.append(x)
    return (pad_sequences(X, maxlen=maxlen))

train_data = vectorize_sentences(Xs, word_idx, final_vocab)
train_data.shape

(1000, 40)

In [ ]:
from keras.layers.core import Activation, Flatten, Dropout, Dense, Merge
from keras.layers import Convolution1D
from keras.layers.pooling import MaxPooling1D
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam, rmsprop